In [187]:
from flask import Flask, request, render_template, make_response
import folium
import geopandas as gpd
import json
import pandas as pd
import numpy as np
import os

In [2]:
data = gpd.read_file(r'C:\Users\conorak\SegregationMaps\Grunnkretser 2020.shp')
data = data.to_crs(epsg=4326)
data['place']=0
for i in range(len(data)):
    if data['grunnkrets'][i][:4]=='0301':
        data['place'][i]=1
data = data[data['place']==1]
data.reset_index(inplace=True)

C:\Users\conorak\AppData\Local\Temp\ipykernel_32524\3811594782.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['place'][i]=1


In [57]:
gk = pd.read_csv(r'C:\Users\conorak\SegregationMaps\529.csv',sep=';',encoding = 'unicode_escape')
data['gk']='0'; data['gkname']='0'
for i in range(len(data)):
    data['gk'][i] = gk['sourceCode'][np.where(gk['targetCode']==int(data['grunnkrets'][i]))[0][0]]
    data['gkname'][i] = gk['sourceName'][np.where(gk['targetCode']==int(data['grunnkrets'][i]))[0][0]]
datagk = data.loc[np.unique(data['gk'],return_index=True)[1]]
datagk.reset_index(inplace=True)

for i in range(len(datagk)):
    poly = data['geometry'].loc[np.where(data['gk']==datagk['gk'][i])]
    datagk['geometry'][i] = poly.unary_union

C:\Users\conorak\AppData\Local\Temp\ipykernel_32524\4119141938.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['gk'][i] = gk['sourceCode'][np.where(gk['targetCode']==int(data['grunnkrets'][i]))[0][0]]
C:\Users\conorak\AppData\Local\Temp\ipykernel_32524\4119141938.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['gkname'][i] = gk['sourceName'][np.where(gk['targetCode']==int(data['grunnkrets'][i]))[0][0]]


In [58]:
edu = pd.read_excel(r'C:\Users\conorak\Food\edu.xls')
rm=[]
for i in range(len(edu)):
    if edu['Utdanning'][i][0]!='D':
        if edu['Utdanning'][i]=='Sentrum':
            continue
        elif edu['Utdanning'][i]=='Marka':
            continue
        rm.append(i)
edu=edu.drop(rm)
edu=edu.drop([112,114]) #remove duplicates
edu=edu.reset_index(drop=True)
edu['DBD']='0'; edu['lowedu']=0.0
for i in range(len(edu)):
    edu['DBD'][i] = edu['Utdanning'][i][9:]
    edu['lowedu'][i] = ((edu['Grunnskole'][i]+edu['Videregående utdanning'][i]+edu['Uoppgitt eller ingen fullført utdanning'][i])/
                     edu['Utdanningsnivå i alt'][i])
edu['DBD'][96]='Sentrum'; edu['DBD'][97] = 'Marka'
datagk['edu'] = 0.0
for i in range(len(datagk)):
    datagk['edu'][i] = edu['lowedu'][np.where(edu['DBD']==datagk['gkname'][i])[0][0]]

C:\Users\conorak\AppData\Local\Temp\ipykernel_32524\1282114965.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edu['DBD'][i] = edu['Utdanning'][i][9:]
C:\Users\conorak\AppData\Local\Temp\ipykernel_32524\1282114965.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edu['lowedu'][i] = ((edu['Grunnskole'][i]+edu['Videregående utdanning'][i]+edu['Uoppgitt eller ingen fullført utdanning'][i])/
C:\Users\conorak\AppData\Local\Temp\ipykernel_32524\1282114965.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/p

In [60]:
emp = pd.read_excel(r'C:\Users\conorak\Food\emp.xls')
rm=[]
for i in range(len(emp)):
    if emp['Oslo i alt'][i][0]!='D':
        if emp['Oslo i alt'][i]=='Sentrum':
            continue
        elif emp['Oslo i alt'][i]=='Marka':
            continue
        rm.append(i)
emp=emp.drop(rm)
# # dis=dis.drop([112,114]) #remove duplicates
emp=emp.reset_index(drop=True)
emp['DBD']='0'
for i in range(len(emp)):
    emp['DBD'][i] = emp['Oslo i alt'][i][9:]
emp['DBD'][100]='Sentrum'; emp['DBD'][101] = 'Marka'
datagk['emp'] = 0.0
for i in range(len(datagk)):
    datagk['emp'][i] = 100-emp[emp.keys()[2]][np.where(emp['DBD']==datagk['gkname'][i])[0][0]]

C:\Users\conorak\AppData\Local\Temp\ipykernel_32524\2032667193.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datagk['emp'][i] = 100-emp[emp.keys()[2]][np.where(emp['DBD']==datagk['gkname'][i])[0][0]]


In [73]:
dis=pd.read_excel(r'C:\Users\conorak\Food\dis.xls')
rm=[]
for i in range(len(dis)):
    if dis['Oslo i alt'][i][0]!='D':
        if dis['Oslo i alt'][i]=='Sentrum':
            continue
        elif dis['Oslo i alt'][i]=='Marka':
            continue
        rm.append(i)
dis=dis.drop(rm)
# dis=dis.drop([112,114]) #remove duplicates
dis=dis.reset_index(drop=True)
dis['DBD']='0'
for i in range(len(dis)):
    dis['DBD'][i] = dis['Oslo i alt'][i][9:]
dis['DBD'][100]='Sentrum'; dis['DBD'][101] = 'Marka'
datagk['dis'] = 0.0
for i in range(len(datagk)):
    datagk['dis'][i] = dis[dis.keys()[1]][np.where(dis['DBD']==datagk['gkname'][i])[0][0]]

C:\Users\conorak\AppData\Local\Temp\ipykernel_32524\253724697.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datagk['dis'][i] = dis[dis.keys()[1]][np.where(dis['DBD']==datagk['gkname'][i])[0][0]]


In [96]:
inc=pd.read_excel(r'C:\Users\conorak\Food\inc.xls')
rm=[]
for i in range(len(inc)):
    if inc['Oslo i alt'][i][0]!='D':
        if inc['Oslo i alt'][i]=='Sentrum':
            continue
        elif inc['Oslo i alt'][i]=='Marka':
            continue
        rm.append(i)
inc=inc.drop(rm)
inc=inc.drop([117,119]) #remove duplicates
inc=inc.reset_index(drop=True)
inc['DBD']='0'
for i in range(len(inc)):
    inc['DBD'][i] = inc['Oslo i alt'][i][9:]
inc['DBD'][100]='Sentrum'; inc['DBD'][101] = 'Marka'
datagk['inc'] = 0.0
for i in range(len(datagk)):
    datagk['inc'][i] = inc[inc.keys()[1]][np.where(inc['DBD']==datagk['gkname'][i])[0][0]]

C:\Users\conorak\AppData\Local\Temp\ipykernel_32524\4189630906.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datagk['inc'][i] = inc[inc.keys()[1]][np.where(inc['DBD']==datagk['gkname'][i])[0][0]]


In [82]:
# app = Flask(__name__)

# @app.route('/', methods = ['GET','POST'])
# def index():
#     m = folium.Map(location=[59.95,10.75],zoom_start=10.4)
#     #folium.GeoJson(data=datagk['geometry']).add_to(m)    
#     #folium.GeoJson(data=data['geometry']).add_to(m)
    
#     if request.method=='POST':
#         user_input = request.form['user_input']
#         #user_input = request.get_json()

#         if user_input == 'Unemployed':
#             folium.Choropleth(
#             geo_data=datagk['geometry'],
#             name="choropleth",
#             data=datagk['emp'],
#             columns=["DBD", "emp"],
#             key_on="feature.id",
#             fill_color="Reds",
#             fill_opacity=0.6,
#             line_opacity=0.2,
#             legend_name="Unemployment Rate (%)").add_to(m)
#         elif user_input == 'Percentage of persons with at most VGS':
#             folium.Choropleth(
#                 geo_data=datagk['geometry'],
#                 name="choropleth",
#                 data=datagk['edu'],
#                 columns=["DBD", "edu"],
#                 key_on="feature.id",
#                 fill_color="Reds",
#                 fill_opacity=0.6,
#                 line_opacity=0.2,
#                 legend_name="Percentage low-education").add_to(m)
#         elif user_input == 'Percentage registered disabled':
#             folium.Choropleth(
#                 geo_data=datagk['geometry'],
#                 name="choropleth",
#                 data=datagk['dis'],
#                 columns=["DBD", "dis"],
#                 key_on="feature.id",
#                 fill_color="Reds",
#                 fill_opacity=0.6,
#                 line_opacity=0.2,
#                 legend_name="Percentage registered disabled").add_to(m)
        
#     else:
#         folium.GeoJson(data=datagk['geometry']).add_to(m)
        
#     m.save('templates/map.html')
#     return render_template("index.html",map=m)

# if __name__ == '__main__':
#     app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Feb/2023 11:48:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2023 11:48:27] "POST /update_map HTTP/1.1" 404 -
127.0.0.1 - - [07/Feb/2023 11:48:28] "POST / HTTP/1.1" 200 -


In [80]:
# app = Flask(__name__)

# @app.route('/update_map', methods=['POST'])
# def update_map():
#     user_input = request.get_json()
#     m = folium.Map(location=[59.95, 10.75], zoom_start=10.4)
    
#     if 'Unemployed' in user_input:
#         folium.Choropleth(
#             geo_data=datagk['geometry'],
#             name="choropleth",
#             data=datagk['emp'],
#             columns=["DBD", "emp"],
#             key_on="feature.id",
#             fill_color="Reds",
#             fill_opacity=0.6,
#             line_opacity=0.2,
#             legend_name="Unemployment Rate (%)").add_to(m)
#     if 'Percentage of persons with at most VGS' in user_input:
#         folium.Choropleth(
#             geo_data=datagk['geometry'],
#             name="choropleth",
#             data=datagk['edu'],
#             columns=["DBD", "edu"],
#             key_on="feature.id",
#             fill_color="Reds",
#             fill_opacity=0.6,
#             line_opacity=0.2,
#             legend_name="Percentage low-education").add_to(m)
#     if 'Percentage registered disabled' in user_input:
#         folium.Choropleth(
#             geo_data=datagk['geometry'],
#             name="choropleth",
#             data=datagk['dis'],
#             columns=["DBD", "dis"],
#             key_on="feature.id",
#             fill_color="Reds",
#             fill_opacity=0.6,
#             line_opacity=0.2,
#             legend_name="Percentage registered disabled").add_to(m)
    
#     m.save("templates/map.html")
#     return render_template("index.html",map=m)

# if __name__ == '__main__':
#     app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Feb/2023 11:45:13] "GET / HTTP/1.1" 404 -


In [191]:
app = Flask(__name__)

first_time=True

@app.route('/', methods = ['GET','POST'])
def index():
    m = folium.Map(location=[59.95,10.75],zoom_start=10.4)
    #folium.GeoJson(data=datagk['geometry']).add_to(m)    
    #folium.GeoJson(data=data['geometry']).add_to(m)
    
    if request.method=='POST':
        #user_input = request.form['user_input']
        user_input = list(request.form.listvalues())[0]
        print(user_input,len(user_input))
        #print(list(request.form.listvalues())[0])
        m = folium.Map(location=[59.95,10.75],zoom_start=10.4)
        #user_input = request.get_json()
        if len(user_input)==1:
            if user_input[0] == 'Unemployed':
                folium.Choropleth(
                    geo_data=datagk['geometry'],
                    name="choropleth",
                    data=datagk['emp'],
                    columns=["DBD", "emp"],
                    key_on="feature.id",
                    fill_color="Reds",
                    fill_opacity=0.6,
                    line_opacity=0.2,
                    legend_name="Unemployment Rate (%)").add_to(m)
            elif user_input[0] == 'Percentage of persons with at most VGS':
                folium.Choropleth(
                    geo_data=datagk['geometry'],
                    name="choropleth",
                    data=datagk['edu'],
                    columns=["DBD", "edu"],
                    key_on="feature.id",
                    fill_color="Reds",
                    fill_opacity=0.6,
                    line_opacity=0.2,
                    legend_name="Percentage low-education").add_to(m)
            elif user_input[0] == 'Percentage registered disabled':
                folium.Choropleth(
                    geo_data=datagk['geometry'],
                    name="choropleth",
                    data=datagk['dis'],
                    columns=["DBD", "dis"],
                    key_on="feature.id",
                    fill_color="Reds",
                    fill_opacity=0.6,
                    line_opacity=0.2,
                    legend_name="Percentage registered disabled").add_to(m)
            elif user_input[0] == 'Percentage low income households':
                folium.Choropleth(
                    geo_data=datagk['geometry'],
                    name="choropleth",
                    data=datagk['inc'],
                    columns=["DBD", "inc"],
                    key_on="feature.id",
                    fill_color="Reds",
                    fill_opacity=0.6,
                    line_opacity=0.2,
                    legend_name="Percentage low income households").add_to(m)
        elif len(user_input)>1:
            labels = ['Unemployed','Percentage of persons with at most VGS','Percentage registered disabled','Percentage low income households']
            lab_idx = ['emp','edu','dis','inc']
            datagk['new'] = 0.0
            legend = ''
            for i in range(len(labels)):
                if labels[i] in user_input:
                    datagk['new'] = datagk['new']+datagk[lab_idx[i]]
                    legend = legend + str(labels[i])+" | "
            datagk['new'] = datagk['new']/len(user_input)
            folium.Choropleth(
                geo_data=datagk['geometry'],
                name="choropleth",
                data=datagk['new'],
                columns=["DBD", "new"],
                key_on="feature.id",
                fill_color="Reds",
                fill_opacity=0.6,
                line_opacity=0.2,
                legend_name=legend).add_to(m)
        else:
            folium.GeoJson(data=datagk['geometry']).add_to(m)
            
        
    else:
        folium.GeoJson(data=datagk['geometry']).add_to(m)
        
    m.save('templates/map.html')
    return render_template("index.html",map=m)

# @app.route('/update_map', methods=['POST'])
# def update_map():
#     global first_time
    
#     #user_input = request.get_json()
#     user_input = request.form['user_input']
#     m = folium.Map(location=[59.95, 10.75], zoom_start=10.4)
#     print(user_input)
#     if len(user_input)==1:
#         if 'Unemployed' in user_input:
#             folium.Choropleth(
#                 geo_data=datagk['geometry'],
#                 name="choropleth",
#                 data=datagk['emp'],
#                 columns=["DBD", "emp"],
#                 key_on="feature.id",
#                 fill_color="Reds",
#                 fill_opacity=0.6,
#                 line_opacity=0.2,
#                 legend_name="Unemployment Rate (%)").add_to(m)
#         elif 'Percentage of persons with at most VGS' in user_input:
#             folium.Choropleth(
#                 geo_data=datagk['geometry'],
#                 name="choropleth",
#                 data=datagk['edu'],
#                 columns=["DBD", "edu"],
#                 key_on="feature.id",
#                 fill_color="Reds",
#                 fill_opacity=0.6,
#                 line_opacity=0.2,
#                 legend_name="Percentage low-education").add_to(m)
#         elif 'Percentage registered disabled' in user_input:
#             folium.Choropleth(
#                 geo_data=datagk['geometry'],
#                 name="choropleth",
#                 data=datagk['dis'],
#                 columns=["DBD", "dis"],
#                 key_on="feature.id",
#                 fill_color="Reds",
#                 fill_opacity=0.6,
#                 line_opacity=0.2,
#                 legend_name="Percentage registered disabled").add_to(m)
#     elif len(user_input)>1:
#         print('Hello')
#         labels = ['Unemployed','Percentage of persons with at most VGS','Percentage registered disabled','Percentage low income households']
#         lab_idx = ['emp','edu','dis','inc']
#         datagk['new'] = 0.0
#         for i in range(len(labels)):
#             if labels[i] in user_input:
#                 datagk['new'] = datagk['new']+datagk[lab_idx[i]]
#         datagk['new'] = datagk['new']/len(user_input)
#         folium.Choropleth(
#                 geo_data=datagk['geometry'],
#                 name="choropleth",
#                 data=datagk['new'],
#                 columns=["DBD", "new"],
#                 key_on="feature.id",
#                 fill_color="Reds",
#                 fill_opacity=0.6,
#                 line_opacity=0.2,
#                 legend_name="Composite score").add_to(m)
    
#     if first_time:
#         first_time=False
    
#     m.save("templates/map.html")
# #     response = make_response(m._repr_html_())
# #     response.headers["Content-Type"] = "text/html"
# #     return response
#     return render_template("index.html",map=m)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Feb/2023 15:06:30] "GET / HTTP/1.1" 200 -


['Unemployed', 'Percentage of persons with at most VGS'] 2


127.0.0.1 - - [07/Feb/2023 15:06:34] "POST / HTTP/1.1" 200 -


['Unemployed'] 1


...

127.0.0.1 - - [07/Feb/2023 15:07:15] "POST / HTTP/1.1" 200 -


['Percentage registered disabled'] 1


127.0.0.1 - - [07/Feb/2023 15:07:17] "POST / HTTP/1.1" 200 -


['Percentage low income households'] 1


127.0.0.1 - - [07/Feb/2023 15:07:20] "POST / HTTP/1.1" 200 -


In [108]:
# app = Flask(__name__)

# first_time=True

# @app.route('/')
# def index():
#     return render_template("index.html",map=m)

# @app.route('/update_map', methods=['POST'])
# def update_map():
#     global first_time,m
#     user_input = request.get_json()
#     m = folium.Map(location=[59.95, 10.75], zoom_start=10.4)
    
#     print(user_input,len(user_input))
    
#     if len(user_input)==1:
#         if 'Unemployed' in user_input:
#             folium.Choropleth(
#                 geo_data=datagk['geometry'],
#                 name="choropleth",
#                 data=datagk['emp'],
#                 columns=["DBD", "emp"],
#                 key_on="feature.id",
#                 fill_color="Reds",
#                 fill_opacity=0.6,
#                 line_opacity=0.2,
#                 legend_name="Unemployment Rate (%)").add_to(m)
#         if 'Percentage of persons with at most VGS' in user_input:
#             folium.Choropleth(
#                 geo_data=datagk['geometry'],
#                 name="choropleth",
#                 data=datagk['edu'],
#                 columns=["DBD", "edu"],
#                 key_on="feature.id",
#                 fill_color="Reds",
#                 fill_opacity=0.6,
#                 line_opacity=0.2,
#                 legend_name="Percentage low-education").add_to(m)
#         if 'Percentage registered disabled' in user_input:
#             folium.Choropleth(
#                 geo_data=datagk['geometry'],
#                 name="choropleth",
#                 data=datagk['dis'],
#                 columns=["DBD", "dis"],
#                 key_on="feature.id",
#                 fill_color="Reds",
#                 fill_opacity=0.6,
#                 line_opacity=0.2,
#                 legend_name="Percentage registered disabled").add_to(m)
#     elif len(user_input)>1:
#         labels = ['Unemployed','Percentage of persons with at most VGS','Percentage registered disabled','Percentage low income households']
#         lab_idx = ['emp','edu','dis','inc']
#         datagk['new'] = 0.0
#         for i in range(len(labels)):
#             if labels[i] in user_input:
#                 datagk['new'] = datagk['new']+datagk[lab_idx[i]]
#         datagk['new'] = datagk['new']/len(user_input)
#         folium.Choropleth(
#                 geo_data=datagk['geometry'],
#                 name="choropleth",
#                 data=datagk['new'],
#                 columns=["DBD", "new"],
#                 key_on="feature.id",
#                 fill_color="Reds",
#                 fill_opacity=0.6,
#                 line_opacity=0.2,
#                 legend_name="Composite score").add_to(m)
    
#     if first_time:
#         first_time=False
    
#     m.save("templates/map.html")
#     return "Map updated"
#     #return render_template("index.html",map=m)

# if __name__ == '__main__':
#     app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Feb/2023 12:41:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2023 12:42:00] "POST /update_map HTTP/1.1" 200 -


{'Unemployed': True} 1


127.0.0.1 - - [07/Feb/2023 12:42:01] "POST / HTTP/1.1" 405 -


In [ ]:
m = folium.Map(location=[59.9,10.75],zoom_start=11)
#folium.plugins.Draw(data,polyline=True).add_to(m)
#folium.GeoJson(data,overlay=True).add_to(m)
#folium.GeoJson(data=data['geometry']).add_to(m)
#folium.GeoJson(data=datagk['geometry']).add_to(m)
folium.Choropleth(
    geo_data=datagk['geometry'],
    name="choropleth",
    data=datagk['emp'],
    columns=["DBD", "emp"],
    key_on="feature.id",
    fill_color="Reds",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(m)
m

In [ ]:
data